In [3]:
import sys
import os 
base_dir = os.path.dirname(os.getcwd())
output_base_dir = os.path.join(base_dir, 'output')
sys.path.append(os.path.join(base_dir, 'source'))

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
# from config import parameters
import os
from six.moves import cPickle as pickle
from LSTMCNN import LSTMCNN

output_time_dir = '20200113-23-56-59'

vocab_npz_filepath = os.path.join(output_base_dir, output_time_dir, 'vocab.npz')
param_pkl_filepath = os.path.join(output_base_dir, output_time_dir, 'model_param.pkl')
model_weights_h5_filepath = os.path.join(output_base_dir, output_time_dir, 'epoch25_181.46.h5')

def vocab_unpack(vocab):
    return vocab['idx2word'], vocab['word2idx'], vocab['idx2char'], vocab['char2idx']

def word_vector(emb_layer, word_idx):
    weights = emb_layer.weights[0]
    return weights.value()[:, word_idx]

Using TensorFlow backend.


In [5]:
vocab_mapping = np.load(vocab_npz_filepath)
idx2word, word2idx, idx2char, char2idx = vocab_unpack(vocab_mapping)
type(word2idx)

numpy.ndarray

In [6]:
list(word2idx.tolist().keys())[10:20]

['활용', '분류', '빅데이터', '처리', '적용', '성능', '알고리즘', '기법', '제안', 'vol']

In [7]:
n_chars = 100
char_vocab_size = min(n_chars, len(idx2char))
char_vec_size = 15
feature_maps = [50,100,150,200,200,200,200]
kernels = [1,2,3,4,5,6,7]
batch_size = 20
seq_length = 35
max_word_l = 100
batch_norm = 0
highway_layers = 2
num_layers = 2
rnn_size = 650
dropout = 0.5
n_words = 30000
word_vocab_size = min(n_words, len(idx2word))
learning_rate = 1
max_grad_norm = 5

In [8]:
model = LSTMCNN(char_vocab_size, char_vec_size, feature_maps, kernels, batch_size, seq_length, max_word_l, batch_norm, highway_layers, num_layers, rnn_size, dropout, word_vocab_size, learning_rate, max_grad_norm)
model.load_weights(model_weights_h5_filepath)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
chars (InputLayer)              (20, 35, 100)        0                                            
__________________________________________________________________________________________________
chars_embedding (Embedding)     (20, 35, 100, 15)    825         chars[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (20, 35, 100, 50)    800         chars_embedding[0][0]            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)    

In [11]:
TARGET_WORD = '활용'
word_vector_target = word_vector(model.layers[-1], word2idx.tolist()[TARGET_WORD])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    word_vector_target = sess.run(word_vector_target)
    print('word_vector.shape: ', word_vector_target.shape)
    print(word_vector_target)

word_vector.shape:  (650,)
[-7.65000843e-03 -4.73922305e-03 -6.32091099e-03 -9.91699379e-03
  3.87320295e-04  1.32416487e-02  4.14037891e-03 -3.24439630e-03
  8.97601061e-03  6.23693876e-03 -8.47961009e-03 -6.98811514e-03
 -2.64802109e-03  1.20685380e-02  1.16835386e-02  1.39552075e-02
 -9.71963070e-03  1.01289507e-02 -4.36392520e-03  1.03849769e-02
  9.13549587e-03  1.26310233e-02  8.29201005e-03 -5.98875154e-03
 -5.58539294e-03  1.08203758e-03  8.87691975e-03 -1.23556964e-02
 -1.00769857e-02  7.86763616e-03  3.71644460e-03  3.42650339e-03
 -3.54485214e-03  6.01756573e-03 -6.81728497e-03  6.31223433e-03
  8.57054442e-03  9.94095206e-03  1.25506874e-02 -8.73196963e-03
  1.38002448e-04 -1.00960564e-02 -9.78046283e-03 -1.39790718e-02
 -8.16203747e-03 -3.86885181e-03 -4.92661260e-04 -1.16775241e-02
  1.63997523e-03  1.00588147e-03 -1.19337477e-02 -1.95412338e-03
 -1.38522685e-02 -9.74490680e-03  1.04622673e-02  2.95157917e-03
  1.05996523e-03  2.71299854e-03 -5.37588261e-04  1.05935875e-0